In [1]:
import pandas as pd
import os
import json
import time
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus.reader.api import *
from nltk.corpus.reader.util import *
from nltk.tokenize import line_tokenize

### Reading Json file

In [2]:
list_one = []
chunksize = 5000
for chunk in pd.read_json('D:\DA\Technocolabs Intern\Final Project\WSB comments\wsbData.json', chunksize=chunksize, lines=True):
    list_one.append(chunk)

In [3]:
len(list_one)

596

In [4]:
list_one[0]

,body,score_hidden,archived,name,author,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,author_flair_text,author_flair_css_class,distinguished
0,[deleted],False,True,t1_chi0ufo,[deleted],0,1400098596,t5_2th52,t3_25jeio,t1_chhzelx,0,1433797242,0,0,chi0ufo,wallstreetbets,0,NaN,NaN,NaN
1,"Lol. Yeah, Welp.",False,False,t1_cl52foo,JamesAQuintero,0,1412888184,t5_2th52,t3_2ikdc5,t1_cl47s92,1,1426604732,0,0,cl52foo,wallstreetbets,1,NaN,NaN,NaN
2,[deleted],False,True,t1_ch5wdhy,[deleted],0,1398860210,t5_2th52,t3_24a77l,t1_ch5diku,1,1433587670,0,0,ch5wdhy,wallstreetbets,1,NaN,NaN,NaN
3,Crypto miners is not a significant enough mark...,False,True,t1_cfj9fzo,Zytran,0,1392824901,t5_2th52,t3_1y409u,t3_1y409u,1,1432465602,0,0,cfj9fzo,wallstreetbets,1,NaN,NaN,NaN
4,Sold a covered call at 560. I almost wish it d...,False,True,t1_ch0jw88,FarmPal,0,1398311440,t5_2th52,t3_23sjde,t3_23sjde,1,1433495365,0,0,ch0jw88,wallstreetbets,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,I agree I think the future is solar and hydrog...,False,True,t1_ch6xd8k,DKSArtwork,0,1398959298,t5_2th52,t3_24dmbo,t3_24dmbo,2,1433605430,0,0,ch6xd8k,wallstreetbets,2,NaN,NaN,NaN
4996,Did you do your DD? \n\n[NQ HQ is in Dallas ju...,False,False,t1_ckx7u62,tacitusk,0,1412138886,t5_2th52,t3_2hxg6m,t1_ckx0q0f,2,1426741504,0,0,ckx7u62,wallstreetbets,2,NaN,NaN,NaN
4997,"If earnings weren't tomorrow, are you happy wi...",False,True,t1_ch4r8cb,j0r3,0,1398741993,t5_2th52,t3_2487jw,t3_2487jw,2,1433567947,0,0,ch4r8cb,wallstreetbets,2,NaN,NaN,NaN
4998,Greed will rape you time and time again. Right...,False,True,t1_cf90pm4,sixgrainnut,0,1391736507,t5_2th52,t3_1x754v,t3_1x754v,3,1432288189,0,0,cf90pm4,wallstreetbets,3,NaN,NaN,NaN


In [6]:
df = pd.concat(list_one, axis=0)

In [7]:
df.tail()

,body,score_hidden,archived,name,author,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,author_flair_text,author_flair_css_class,distinguished
2979126,true tho in't it,NaN,NaN,NaN,doom_pork,NaN,1485678518,t5_2th52,t3_5qprhh,t1_dd1ck9r,1,1.486528e+09,0,0,dd1u8rw,wallstreetbets,NaN,NaN,NaN,NaN
2979127,Gold is a key element in electronics. The bot ...,NaN,NaN,NaN,WolfOfSesameSt,NaN,1484101062,t5_2th52,t3_5n59w2,t1_dc8q7k9,1,1.485948e+09,0,0,dc9oyiy,wallstreetbets,NaN,NaN,NaN,NaN
2979128,[deleted],NaN,NaN,NaN,[deleted],NaN,1485545272,t5_2th52,t3_5qegzc,t1_dcyseir,1,1.486490e+09,0,0,dczmotp,wallstreetbets,NaN,NaN,NaN,NaN
2979129,Listen up kids as I tell you how I earned my f...,NaN,NaN,NaN,outofsync42,NaN,1484084598,t5_2th52,t3_5n79ha,t1_dc9bjck,9,1.485942e+09,0,0,dc9cc39,wallstreetbets,NaN,AMZN wins gg no re,NaN,NaN
2979130,sometimes you should sell with a loss. Jnug ha...,NaN,NaN,NaN,TheCynicalDick,NaN,1485360569,t5_2th52,t3_5q3rlf,t1_dcw27ke,4,1.486428e+09,0,0,dcw306m,wallstreetbets,NaN,NaN,NaN,NaN


In [8]:
df.shape

(2979131, 20)

In [9]:
df.isnull().sum()

body                            0
score_hidden              2356498
archived                  1680092
name                      2867023
author                          0
downs                     2867023
created_utc                     0
subreddit_id                    0
link_id                         0
parent_id                       0
score                           0
retrieved_on                  353
controversiality                0
gilded                          0
id                              0
subreddit                       0
ups                       2384940
author_flair_text         2573430
author_flair_css_class    2955357
distinguished             2953192
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979131 entries, 0 to 2979130
Data columns (total 20 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   body                    object 
 1   score_hidden            object 
 2   archived                object 
 3   name                    object 
 4   author                  object 
 5   downs                   float64
 6   created_utc             int64  
 7   subreddit_id            object 
 8   link_id                 object 
 9   parent_id               object 
 10  score                   int64  
 11  retrieved_on            float64
 12  controversiality        int64  
 13  gilded                  int64  
 14  id                      object 
 15  subreddit               object 
 16  ups                     float64
 17  author_flair_text       object 
 18  author_flair_css_class  object 
 19  distinguished           object 
dtypes: float64(3), int64(4), object(13)
memory usage: 454.6+ MB


In [11]:
comments = df['body']

In [12]:
comments.head(3)

0           [deleted]
1    Lol. Yeah, Welp.
2           [deleted]
Name: body, dtype: object

In [13]:
comments = comments.dropna()

# Text Processing

In [14]:
#lower casing
comments = comments.str.lower()

In [15]:
#Removal of Punctuations
import string
punct = string.punctuation
def remove_punctuation(comments):
    return comments.translate(str.maketrans('', '', punct))

In [16]:
comments_ponct = comments.apply(lambda comments: remove_punctuation(comments))

In [17]:
#Removal of Stop Words
stop_words = stopwords.words('english')
from sklearn.feature_extraction import text
my_stop_words = text.ENGLISH_STOP_WORDS
stop_words.extend(my_stop_words)

In [18]:
def remove_stopwords(comments_ponct):
    return " ".join([word for word in str(comments_ponct).split() if word not in stop_words])
comments_wo_stopwords = comments_ponct.apply(lambda comments_ponct: remove_stopwords(comments_ponct))

In [19]:
comments_wo_stopwords = ''.join(map(str, comments_wo_stopwords))

In [ ]:
comment_words = word_tokenize(comments_wo_stopwords)
comment_words

In [22]:
word_counts = nltk.FreqDist(comment_words)
word_counts

FreqDist({'’': 224050, 'like': 203504, 'im': 149011, 'dont': 121556, 'buy': 119592, 'money': 118544, 'going': 114408, 'calls': 105917, 'think': 105565, 'market': 97111, ...})

In [24]:
word_counts.most_common(25)

[('’', 224050),
 ('like', 203504),
 ('im', 149011),
 ('dont', 121556),
 ('buy', 119592),
 ('money', 118544),
 ('going', 114408),
 ('calls', 105917),
 ('think', 105565),
 ('market', 97111),
 ('good', 97002),
 ('people', 96101),
 ('stock', 91857),
 ('time', 90211),
 ('make', 86808),
 ('shit', 83631),
 ('know', 75952),
 ('sell', 74421),
 ('s', 72941),
 ('got', 71032),
 ('day', 69039),
 ('right', 68827),
 ('long', 68100),
 ('puts', 67672),
 ('t', 62789)]

In [ ]:
wc = WordCloud(
    background_color = 'black',
    stopwords = stop_words,
    height = 700,
    width = 1000
    )
wc.generate(comments_wo_stopwords)
plt.imshow(wc)
plt.axis("off")
plt.show()

pandas.core.series.Series